In [1]:
import base58
import binascii
import ecdsa
import hashlib
import json
import os
import subprocess
import time

def dSHA256(data):
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(s):
    '''sha256 followed by ripemd160'''
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def privkey_to_pubkey(privkey):
    signing_key = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1) # Don't forget to specify the curve
    verifying_key = signing_key.get_verifying_key()

    # Use this code block if the address you gave corresponds to the compressed public key
    x_cor = bytes.fromhex(verifying_key.to_string().hex())[:32] # The first 32 bytes are the x coordinate
    y_cor = bytes.fromhex(verifying_key.to_string().hex())[32:] # The last 32 bytes are the y coordinate
    if int.from_bytes(y_cor, byteorder="big", signed=True) % 2 == 0: # We need to turn the y_cor into a number.
        public_key = bytes.fromhex("02" + x_cor.hex())
    else:
        public_key = bytes.fromhex("03" + x_cor.hex())
    return public_key

In [2]:
# Functions related to generating bitcoin addresses

def encode_base58(s):
    BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    count = 0
    for c in s:  
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes(s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:  
        num, mod = divmod(num, 58)
        result = BASE58_ALPHABET[mod] + result
    return prefix + result

def encode_base58_checksum(b):
    return encode_base58(b + dSHA256(b)[:4])

# https://en.bitcoin.it/wiki/Wallet_import_format
def privkey_to_wif(privkey, compressed_pubkey, testnet):
    if testnet:
        prefix = b"\xEF"
    else:
        prefix = b"\x80"
    
    # if the privkey will correspond to a compressed public key
    if compressed_pubkey: 
        extended = prefix + privkey + b"\x01"
    else:
        extended = prefix + privkey 
        
    extendedchecksum = extended + dSHA256(extended)[:4]
    wif = encode_base58(extendedchecksum)
    
    return wif

# https://learnmeabitcoin.com/guide/wif
def wif_to_privkey(private_key_WIF):
    private_key_full = base58.b58decode(private_key_WIF)
    
    # If the WIF encoding includes the optional "01" byte for compressed privKey,
    # do not include it in the final output.
    if len(private_key_full) == 38:
        private_key = private_key_full[1:-5] 
        print("compressed pubkey")
    else:
        private_key = private_key_full[1:-4] 
        print("not compressed pubkey")
    return private_key


def pk_to_p2pkh(compressed, testnet):
    '''Returns the address string'''
    pk_hash = hash160(compressed)
    if testnet:
        prefix = b'\x6f'
    else:
        prefix = b'\x00'
    return encode_base58_checksum(prefix + pk_hash)

def pk_to_p2sh_p2wpkh(compressed, testnet):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    rs_hash = hash160(redeemScript)
    if testnet:
        prefix = b"\xc4"
    else:
        prefix = b"\x05"
    return encode_base58_checksum(prefix + rs_hash)

def wif_to_addresses(wif):
#     wif = "cUy9rC6wteKizfu1fgP2abKUWTkJxjqKp2fba91FkU332CFHo6ix"
    privkey = wif_to_privkey(wif)
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("WIF Private key: " + wif)
    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)
    
def privkey_to_addresses(privkey):
    # privkey = bytes.fromhex("AF933A6C602069F1CBC85990DF087714D7E86DF0D0E48398B7D8953E1F03534A")
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, testnet = True)
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)

# Bitcoin core regtest tx tester
This notebook will start up bitcoind in regtest mode, create transactions using python scripts, and broadcast them to validate that they work. Four transactions are tested here:<br><br>

- tx1 initial tx to fund customer. 1 output 
    - Send 11.3 btc to a P2SH-P2WPKH address. This will be used to fund the customer.
- tx2 funding tx with change. 2 outputs
    - 10.2 btc to an escrow (2-of-2 multisig) 
    - 1 btc to a change address
- tx3 merch close. 1 output
    - 11.3 btc to p2wsh with two ways to spend from it:
        - 2 of 2 multisig signed by cust_pk and merch_pk (cust-close-tx)
        - to_self_delay to merch_close_pk (merch-claim-tx)
- tx4 cust close. 2 output
    - 8 btc to p2wsh with two ways to spend from it:
        - to_self_delay to cust_close_pk (cust-claim-tx)
        - using revocation lock to send to merch_close_pk 
    - 2 btc to merch_close_pk (immediately available)
    - 0 btc OP_RETURN 
- tx 5 merch dispute. 1 output
    - 7.9 btc to merch pubkey

Funding private key is used to fund the escrow tx. <br> 
Merch private key and Cust private key are used to generate the pubkeys in the escrow tx. <br>
The Cust private key is also used for the P2WPKH change output.

In [16]:
funding_privkey_hex = "8911111111111111111111111111111111111111111111111111111111111111"
funding_pubkey_hex = privkey_to_pubkey(bytes.fromhex(funding_privkey_hex)).hex()

merch_privkey_hex = "3711111111111111111111111111111111111111111111111111111111111111"
merch_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_privkey_hex)).hex()

cust_privkey_hex = "7911111111111111111111111111111111111111111111111111111111111111"
cust_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_privkey_hex)).hex()

cust_close_privkey_hex = "7711111111111111111111111111111111111111111111111111111111111111"
cust_close_pubkey_hex = privkey_to_pubkey(bytes.fromhex(cust_close_privkey_hex)).hex()

change_privkey_hex = "6911111111111111111111111111111111111111111111111111111111111111"
change_pubkey_hex = privkey_to_pubkey(bytes.fromhex(change_privkey_hex)).hex()

merch_close_privkey_hex = "3911111111111111111111111111111111111111111111111111111111111111"
merch_close_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_close_privkey_hex)).hex()

merch_disp_privkey_hex = "3111111111111111111111111111111111111111111111111111111111111111"
merch_disp_pubkey_hex = privkey_to_pubkey(bytes.fromhex(merch_disp_privkey_hex)).hex()

revocation_secret_hex = "4011111111111111111111111111111111111111111111111111111111111111"

RL = hashlib.sha256(bytes.fromhex(revocation_secret_hex)).digest()
revocation_lock_hex = RL.hex()

merch_close_pubkey_hex


'02f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce90'

Start up regtest mode, delete any history so we are starting from scratch.
Mine 101 blocks

In [24]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(1.5) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# start up bitcoind in regtest mode
os.system("bitcoind -regtest -daemon -minrelaytxfee=0")
time.sleep(1.5)

# generate 101 blocks so we can fund transactions
os.system("bitcoin-cli -regtest generate 101")
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")

print("blockcount: " + str(blockcount))

blockcount: 101


Generate base58 address for the escrow funder

In [25]:
# Generate p2sh-p2wpkh address to fund the escrow funder
privkey = bytes.fromhex(funding_privkey_hex)
public_key = privkey_to_pubkey(privkey)
p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, testnet = True)

# print("Private key: " + privkey.hex())
# print("Public key: " + public_key.hex())
print("Address: " + p2sh_p2wpkh_address)

Address: 2N1jKbzrUkf583t9mM9ePN3rkqdCwryKcf9


Send btc to the escrow funder

In [26]:
txid_1 = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + p2sh_p2wpkh_address + " 3.0")
print(txid_1)

5bf6f7f7c8cbfe9ffb79fac1b9e540cc4087fe10b6d0a670491f3ce5f233b2b7


In [27]:
# Find which output index the btc was sent to
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_1)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
d = json.loads(decoded)
# print(decoded)

if d["vout"][0]["scriptPubKey"]["addresses"][0] == p2sh_p2wpkh_address:
    index = 0
else:
    index = 1
print("index: " + str(index))

index: 1


In [28]:
os.system("bitcoin-cli -regtest generate 1");

Generate raw escrow funding transaction

In [29]:
raw_escrow_tx = subprocess.getoutput("python funding_tx_with_changev2.py" 
                        + " --txid " + txid_1
                        + " --index " + str(index)
                        + " --input_amount_btc " + "3.0"
                        + " --funding_privkey " + funding_privkey_hex
                        + " --escrow_value_btc " + "2"
                        + " --cust_pubkey " + cust_pubkey_hex
                        + " --merch_pubkey " + merch_pubkey_hex
                        + " --cust_change_value_btc " + "1"
                        + " --cust_change_pubkey " + change_pubkey_hex)
print(raw_escrow_tx)

02000000000101b7b233f2e53c1f4970a6d0b610fe8740cc40e5b9c1fa79fb9ffecbc8f7f7f65b0100000017160014fc5cc2b7bdcb852b225434d133769a551486950affffffff0200c2eb0b000000002200209a7574691624d1d6f82e1b81a8a16f20eb68aa22cc850aef075be99f64cac7bc00e1f50500000000160014f3a8a4335c0ef84806d93315e56965f13d522e5f02483045022100c624e6e8e1be92aea45629803391f1dce4c2923753d3187adb8a9c969b4ae57f022005830e42e6858b3bd2aafb661963455358d3612d969913f10a851aa098aa60b5012103e2aa89cce89e9b2d6f09b20a2096226328f114a4ca62e6ea416b4d7c4573086e00000000


Broadcast escrow funding transaction

In [30]:
escrow_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_escrow_tx + " true")
# "true" flag means we are okay with absurdly high tx fee
print(escrow_txid)

2b9e1462007cf04b35787c5ccefcf43e9477ca44c7776219de18299e30df8503


In [31]:
raw_escrow_tx1 = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + escrow_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_escrow_tx1)
# print(decoded)

In [32]:
os.system("bitcoin-cli -regtest generate 1");

Create transaction spending from the change output of the escrow transaction

In [40]:
raw_merch_close_tx = subprocess.getoutput("python merch_closev2.py"        
                + " --txid_str " + escrow_txid
                + " --index " + "0"
                + " --input_amount_btc " + "2"
                + " --cust_privkey " + cust_privkey_hex 
                + " --merch_privkey " + merch_privkey_hex      
                + " --output_value_btc " + "2"
                + " --merch_payout_pubkey " + merch_close_pubkey_hex   
                + " --to_self_delay " + "05cf") # number of blocks to delay to-self output       
print(raw_merch_close_tx)

# parser.add_argument("--txid_str", "-tx", help="txid of input as string")
# parser.add_argument("--index", "-ind", help="index of outpoint")
# parser.add_argument("--input_amount_btc", "-a", help="amount of btc held by the previous outpoint")
# parser.add_argument("--cust_privkey", "-csk", help="private key of customer for escrow")
# parser.add_argument("--merch_privkey", "-msk", help="private key of merchant for escrow")
# parser.add_argument("--output_value_btc", "-o", help="btc to output")
# parser.add_argument("--merch_payout_pubkey", "-mcpk", help="public key of merchant close to-self output")
# parser.add_argument("--to_self_delay", "-tsd", help="to_self_delay (in unit of blocks) for the merchant's to-self output")

decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_merch_close_tx)
# print(decoded)

020000000001010385df309e2918de196277c744ca77943ef4fcce5c7c78354bf07c0062149e2b0000000000ffffffff0100c2eb0b00000000220020623e1ac60cebb9158bdabaf22ebac2e53805c0462e61218ef9aab1328c8fe627040047304402204935fbcd6d00fb1f2876b1a8db6e4041559e1b940e38f154ea8b485f3e0c0cc802202e7fd24c2e226f96405b88c6f0163938daed66503e064cd4dc93f03f37db9c600147304402201672b33badc3c937aca41fbaf8ecac73fd48555938c54482411305fa639481ae02206ddf0a98f86eb5c152b864fb45c8136e52902a82e03196373e5133a90f79de76014752210342da23a1de903cd7a141a99b5e8051abfcd4d2d1b3c2112bac5c8997d9f12a002103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae00000000


In [41]:
merch_close_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_close_tx + " true")
print(merch_close_txid)

1f40ca0870cfa67cbca6d523a9956159f78ee8bc58b153d60d07d2c5d063f639


In [42]:
merch_close_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_close_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_close_tx)
print(decoded)

{
  "txid": "1f40ca0870cfa67cbca6d523a9956159f78ee8bc58b153d60d07d2c5d063f639",
  "hash": "08868bf3ad1fd64c4b7bd84b54a2b44d696f80b91b34d30839ace7ae89a13a61",
  "version": 2,
  "size": 314,
  "vsize": 149,
  "weight": 596,
  "locktime": 0,
  "vin": [
    {
      "txid": "2b9e1462007cf04b35787c5ccefcf43e9477ca44c7776219de18299e30df8503",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "304402204935fbcd6d00fb1f2876b1a8db6e4041559e1b940e38f154ea8b485f3e0c0cc802202e7fd24c2e226f96405b88c6f0163938daed66503e064cd4dc93f03f37db9c6001",
        "304402201672b33badc3c937aca41fbaf8ecac73fd48555938c54482411305fa639481ae02206ddf0a98f86eb5c152b864fb45c8136e52902a82e03196373e5133a90f79de7601",
        "52210342da23a1de903cd7a141a99b5e8051abfcd4d2d1b3c2112bac5c8997d9f12a002103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 

In [43]:
os.system("bitcoin-cli -regtest generate 1");

# cust-close from merch-close

In [44]:
raw_cust_close_tx = subprocess.getoutput("python cust_closev2.py"    
                + " --spend_from " + "merch-close"  
                + " --txid_str " + merch_close_txid
                + " --index " + "0"    
                + " --input_amount_btc " + "2"
                + " --cust_privkey " + cust_privkey_hex 
                + " --merch_privkey " + merch_privkey_hex  
                + " --cust_script_value_btc " + "1"
                + " --cust_payout_pubkey " + cust_close_pubkey_hex     
                + " --merch_payout_value_btc " + "1"    
                + " --merch_payout_pubkey " + merch_close_pubkey_hex   
                + " --revocation_lock " + revocation_lock_hex      
                + " --merch_dispute_pubkey " + merch_disp_pubkey_hex     
                + " --to_self_delay " + "05cf") # number of blocks to delay to-self output 

print(raw_cust_close_tx)

decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_cust_close_tx)
print(decoded)

0200000000010139f663d0c5d2070dd653b158bce88ef7596195a923d5a6bc7ca6cf7008ca401f0000000000ffffffff0300e1f5050000000022002067cb20e705c4eb4363194a74d2f743afc1c9ee3cd741d45e21268b16add04f8b00e1f5050000000016001443e9e81bc632ad9cad48fc23f800021c5769a0630000000000000000436a41f8345a21a55dc665b65c8dcfb49488b8e4f337d5c9bb843603f7222a892ce94103195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd7686410500483045022100fffe04b9af852be1593be13471b49466b2128310a83fc91b2ccfa2b8febcbab8022005e4e03f280bf9ca6a7b015bd9655be12e638965503979428b363deb03d242ba01483045022100f3161152f72c23af137883e56feea2d7c5f784183a653155783246a7dd1419d502204b221ded2166adca9bbcac1283b13186b113cb0c3e7b7332c86903b4dcd9ffeb010101726352210342da23a1de903cd7a141a99b5e8051abfcd4d2d1b3c2112bac5c8997d9f12a002103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae6702cf05b2752102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99c1437e6570643ce90ac6800000000
{
  "txid": "59c8018ce838edc3a1ece1b80dc6d41aa3c4b97d290baaee31

In [45]:
cust_close_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_cust_close_tx + " true")
print(cust_close_txid)

59c8018ce838edc3a1ece1b80dc6d41aa3c4b97d290baaee315359dd56dfbe1d


In [46]:
cust_close_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + cust_close_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + cust_close_tx)
print(decoded)

{
  "txid": "59c8018ce838edc3a1ece1b80dc6d41aa3c4b97d290baaee315359dd56dfbe1d",
  "hash": "7cc6880f6f62d9942ee205dba867b5bbabc643f3a35d07bcf9cfa886094279e3",
  "version": 2,
  "size": 468,
  "vsize": 268,
  "weight": 1071,
  "locktime": 0,
  "vin": [
    {
      "txid": "1f40ca0870cfa67cbca6d523a9956159f78ee8bc58b153d60d07d2c5d063f639",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "3045022100fffe04b9af852be1593be13471b49466b2128310a83fc91b2ccfa2b8febcbab8022005e4e03f280bf9ca6a7b015bd9655be12e638965503979428b363deb03d242ba01",
        "3045022100f3161152f72c23af137883e56feea2d7c5f784183a653155783246a7dd1419d502204b221ded2166adca9bbcac1283b13186b113cb0c3e7b7332c86903b4dcd9ffeb01",
        "01",
        "6352210342da23a1de903cd7a141a99b5e8051abfcd4d2d1b3c2112bac5c8997d9f12a002103fc43b44cd953c7b92726ebefe482a272538c7e40fdcde5994a62841525afa8d752ae6702cf05b2752102f3d17ca1ac6dcf42b0297a71abb87f79dfa2c66278cbb99

In [28]:
os.system("bitcoin-cli -regtest generate 1");

In [29]:
raw_merch_dispute_tx = subprocess.getoutput("python bitcoin/merch_dispute.py"   
                + " --merch_disp_privkey " + merch_disp_privkey_hex 
                + " --revocation_lock " + revocation_lock_hex    
                + " --revocation_secret " + revocation_secret_hex 
                + " --cust_close_pubkey " + cust_close_pubkey_hex   
                + " --output_pubkey " + "024596d7b33733c28101dbc6c85901dffaed0cdac63ab0b2ea141217d1990ad4b1"    
                + " --to_self_delay " + "05cf" # number of blocks to delay to-self output         
                + " --txid " + cust_close_txid
                + " --index " + "0"
                + " --amount_btc " + "8"
                + " --output_btc " + "7.9")

print(raw_merch_dispute_tx)

decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_merch_dispute_tx)
print(decoded)

02000000000101a6d2054a33986ce2e6ac41c950a615166de1b18e0f6fb44ebbaf874d705495240000000000ffffffff018071162f00000000160014b7cfef435e3701fdedb7a11164ae44d561698bb904483045022100dc86093122bc346e46414b4e0dc0dcf10d85391750fe15fe0617a6016ca8c78c02200f4c40fe6d0a69df4e0abda837752eaa6fbe011a1ef51503256bbc33f924bed60120401111111111111111111111111111111111111111111111111111111111111101017063a820f8345a21a55dc665b65c8dcfb49488b8e4f337d5c9bb843603f7222a892ce94188210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c6702cf05b2752103195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd76864168ac00000000
{
  "txid": "e8ba0132f36836b1c8f3dcf297d61d04a6dfd7d09f2a1a7b00a84a456252c3d1",
  "hash": "93656712adf32f4ee35a1cd17bde498809e155d82e2266be98706b2acd5bf59b",
  "version": 2,
  "size": 306,
  "vsize": 138,
  "weight": 552,
  "locktime": 0,
  "vin": [
    {
      "txid": "249554704d87afbb4eb46f0f8eb1e16d1615a650c941ace6e26c98334a05d2a6",
      "vout": 0,
      "scriptSig": {
        "as

In [47]:
merch_dispute_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + raw_merch_dispute_tx + " true")
print(merch_dispute_txid)

04574738d62cd8e990dd97413d5d853a50313777e754488e3dde8b26d475216a


In [48]:
merch_dispute_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + merch_dispute_txid)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + merch_dispute_tx)
print(decoded)

{
  "txid": "04574738d62cd8e990dd97413d5d853a50313777e754488e3dde8b26d475216a",
  "hash": "eec20bb6c973070af00dc6e58cb09d146ada655c3fa07d3b7f2687c4f53e7f75",
  "version": 2,
  "size": 305,
  "vsize": 138,
  "weight": 551,
  "locktime": 0,
  "vin": [
    {
      "txid": "3d167d53b188f5f37ee60bc41fed317e0c90ea72b98baa515b7b94fbf837805b",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "304402206c51e20a5aa8194c93cee6a569bc21d1a36263169a35803f2b671a144522694c02203036446aebf6e85bd9710717801cce58444aad723e3ec6a41e97a32935018a9901",
        "4011111111111111111111111111111111111111111111111111111111111111",
        "01",
        "63a820f8345a21a55dc665b65c8dcfb49488b8e4f337d5c9bb843603f7222a892ce94188210253be79afe84fd9342c1f52024379b6da6299ea98844aee23838e8e678a765f7c6702cf05b2752103195e272df2310ded35f9958fd0c2847bf73b5b429a716c005d465009bd76864168ac"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "va

In [24]:
os.system("bitcoin-cli -regtest stop")
time.sleep(1.5) 